In [21]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [22]:
!pip install pytorch-warmup

In [23]:
import math, torch, torchaudio
import soundfile
import torch.nn as nn
import torch.nn.functional as F
import os, numpy
from sklearn import metrics
from operator import itemgetter
import glob, numpy, os, random, soundfile, torch
from scipy import signal
import time
import tqdm
import pytorch_warmup as warmup
from torch.utils.tensorboard import SummaryWriter

In [25]:
class test_loader(object):
    def __init__(self, train_list, train_path, musan_path, rir_path, num_frames, **kwargs):
        self.train_path = train_path
        self.num_frames = num_frames
        self.data_list  = []
        self.data_label = []
        lines = open(train_list).read().splitlines()
        dictkeys = list(set([x.split()[0] for x in lines]))
        dictkeys.sort()
        dictkeys = { key : ii for ii, key in enumerate(dictkeys) }
        for index, line in enumerate(lines):
            speaker_label = dictkeys[line.split()[0]]
            file_names = line.split()[1:]  # Get the file names from the line
            file_paths = []
            for file_name in file_names:
                full_file_path = os.path.join(train_path, file_name)
                file_paths.append(full_file_path)
            self.data_label.append(speaker_label)
            self.data_list.append(file_paths)  # Append both file paths

    def __getitem__(self, index):
        try:
            # Read the utterance and randomly select the segment
            audio1, sr1 = soundfile.read(self.data_list[index][0])        
            length = self.num_frames * 160 + 240
            if audio1.shape[0] <= length:
                shortage = length - audio1.shape[0]
                audio1 = numpy.pad(audio1, (0, shortage), 'wrap')
            start_frame = numpy.int64(random.random()*(audio1.shape[0]-length))
            audio1 = audio1[start_frame:start_frame + length]
            #audio1 = numpy.stack([audio1], axis=0).astype(numpy.float32)

            # Read the second utterance and randomly select the segment
            audio2, sr2 = soundfile.read(self.data_list[index][1])        
            if audio2.shape[0] <= length:
                shortage = length - audio2.shape[0]
                audio2 = numpy.pad(audio2, (0, shortage), 'wrap')
            start_frame = numpy.int64(random.random()*(audio2.shape[0]-length))
            audio2 = audio2[start_frame:start_frame + length]
            #audio2 = numpy.stack([audio2], axis=0).astype(numpy.float32)

            # Concatenate the two audio files along a new dimension
            concatenated_audio = numpy.stack([audio1, audio2], axis=0).astype(numpy.float32)
            # Convert concatenated_audio and label to tensors
           
            return torch.tensor(concatenated_audio),(self.data_label[index])
        
        except Exception as e:
#         # If an error occurs (e.g., file not found), print a warning and return None
#         print(f"Error processing file {self.data_list[index]}: {e}")
            length = self.num_frames * 160 + 240
            return torch.zeros((2, length)), self.data_label[index]
        

    def __len__(self):
        return len(self.data_list)

   

In [26]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import roc_curve

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


testloader = test_loader("/kaggle/input/voxceleb-h/voxceleb_h.txt", "/kaggle/input/voxceleb1train/wav", None, None, 300)
testLoader = DataLoader(testloader, batch_size=128, shuffle=False, num_workers=10) 

Using device: cuda


In [16]:
num_files = len(testloader)
print("Number of files in testloader:", num_files)

Number of files in testloader: 552536


In [27]:
#testloader = test_loader("/kaggle/input/voxceleb-h/voxceleb_h.txt", "/kaggle/input/voxceleb1train/wav", None, None, 300)
iteration_count = 0
for batch in testloader:
    inputs = batch[0]  # Assuming inputs are at index 0
    target = batch[1]  # Assuming targets are at index 1
    if inputs is None or target is None:
        continue
    # Your code here

   
    print("Iteration:", iteration_count)
    iteration_count += 1
    # Your remaining code here

    print("inputs:", inputs)
    print("Target:", target)
    
    if iteration_count == 10:
        break
    
# Input: tensor([ 0.0028,  0.0027,  0.0027,  ..., -0.0038, -0.0066, -0.0106])
# Input shape: torch.Size([48240])
# Target: 1

Iteration: 0
inputs: tensor([[ 0.0007, -0.0525, -0.0895,  ...,  0.0495,  0.0458,  0.0446],
        [-0.1032, -0.0710, -0.0002,  ..., -0.0438, -0.0700, -0.1289]])
Target: 1
Iteration: 1
inputs: tensor([[-0.0132,  0.0008,  0.0080,  ..., -0.0184, -0.0343, -0.0600],
        [ 0.0128,  0.0123,  0.0106,  ...,  0.0316,  0.0348,  0.0371]])
Target: 0
Iteration: 2
inputs: tensor([[-0.0153, -0.0112, -0.0045,  ...,  0.0022,  0.0274, -0.0385],
        [-0.0028, -0.0043, -0.0054,  ...,  0.0027,  0.0048,  0.0058]])
Target: 1
Iteration: 3
inputs: tensor([[-0.0013, -0.0018, -0.0014,  ...,  0.0154,  0.0181,  0.0191],
        [-0.0009, -0.0012, -0.0017,  ...,  0.0009, -0.0008, -0.0016]])
Target: 0
Iteration: 4
inputs: tensor([[-0.0367, -0.0519, -0.0566,  ...,  0.0062,  0.0053,  0.0048],
        [-0.0208, -0.0199, -0.0186,  ...,  0.0053,  0.0101,  0.0155]])
Target: 1
Iteration: 5
inputs: tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
Target: 0
Iteration: 6
inputs: tensor(

In [18]:
## Model finetuned on voxceleb1 dataset

#microsoft/unispeech-sat-base-sv
#microsoft/wavlm-base-sv
#microsoft/wavlm-base-plus-sv

In [28]:
import torch
import numpy as np
from sklearn.metrics import roc_curve

from transformers import Wav2Vec2FeatureExtractor, WavLMForXVector

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained('microsoft/wavlm-base-sv')
model = WavLMForXVector.from_pretrained('microsoft/wavlm-base-sv')


# Move model and feature extractor to CUDA if available
model.to(device)

Some weights of the model checkpoint at microsoft/wavlm-base-sv were not used when initializing WavLMForXVector: ['wavlm.encoder.pos_conv_embed.conv.weight_g', 'wavlm.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing WavLMForXVector from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WavLMForXVector from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of WavLMForXVector were not initialized from the model checkpoint at microsoft/wavlm-base-sv and are newly initialized: ['wavlm.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wavlm.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream

WavLMForXVector(
  (wavlm): WavLMModel(
    (feature_extractor): WavLMFeatureEncoder(
      (conv_layers): ModuleList(
        (0): WavLMGroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x WavLMNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x WavLMNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): WavLMFeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): WavLMEncoder(
    

In [30]:
import torch
import numpy as np
from sklearn.metrics import roc_curve

with torch.no_grad():
    similarities = []
    labels = []
    for inputs, target in testLoader:
        #print("Inputs:", inputs)
        #print("Target:", target)
        if inputs is None or target is None:
            continue
            
        # Extract audio1 and audio2 from inputs
        audio1 = inputs[:, 0]  
        audio2 = inputs[:, 1]  
        
        # Pass the input tensors to the model
        audio1 = torch.tensor(audio1)
        audio2 = torch.tensor(audio2)
        #print("audio1 shape", audio1.shape)
        
        # Extract embeddings
        audio1 = feature_extractor(audio1, return_tensors="pt", sampling_rate=16000)
        audio1 = audio1.input_values.squeeze(0) 
       
        audio2 = feature_extractor(audio2, return_tensors="pt", sampling_rate=16000)
        audio2 = audio2.input_values.squeeze(0) 
        
        audio1 = audio1.clone().detach().to(device)
        audio2 = audio2.clone().detach().to(device)
        
        # Get embeddings for audio1
        #print("audio1 shape after embedding extraction", audio1.shape)
        outputs_audio1 = model(input_values=audio1)
        embeddings_audio1 = outputs_audio1.embeddings
        embeddings_audio1 = torch.nn.functional.normalize(embeddings_audio1, dim=-1).cpu()
        
        # Get embeddings for audio2
        outputs_audio2 = model(input_values=audio2)
        embeddings_audio2 = outputs_audio2.embeddings
        embeddings_audio2 = torch.nn.functional.normalize(embeddings_audio2, dim=-1).cpu()
        
        # Calculate cosine similarity between embeddings
        similarity = torch.nn.functional.cosine_similarity(embeddings_audio1, embeddings_audio2, dim=-1)
        
#         print("Similarity:", similarity)
#         print("Label:", target)
        
        similarities.extend(similarity.tolist())
        labels.extend(target.tolist())  # Append the tensor itself, not its item

# Convert lists to numpy arrays
similarities = np.array(similarities)
labels = np.array(labels)

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(labels, similarities, pos_label=1)

# Find EER
eer_threshold = thresholds[np.argmin(np.abs(fpr - (1 - tpr)))]
eer = (fpr[np.argmin(np.abs(fpr - (1 - tpr)))] + (1 - tpr)[np.argmin(np.abs(fpr - (1 - tpr)))]) / 2

print("Equal Error Rate (EER): {:.2f}%".format(eer * 100))


/tmp/ipykernel_34/1011509741.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  audio1 = torch.tensor(audio1)
/tmp/ipykernel_34/1011509741.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  audio2 = torch.tensor(audio2)


Equal Error Rate (EER): 14.85%


####